In [1]:
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()

In [2]:
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

In [3]:
tokenize('how are you ha ha haha')

['how', 'are', 'you', 'ha', 'ha', 'haha']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('HOW')

'how'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words),dtype=np.float32)
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['how', 'are', 'you', 'ha', 'ha', 'haha'],['lol','how','are','you'])

array([0., 1., 1., 1.], dtype=float32)

In [8]:
def make_submission(model):
    model.eval()
    data = pd.read_csv('./data/test.csv')
    ids = data['id']
    data = data['text']
    new_data = []
    for sentence in data:
        bag = bag_of_words(sentence,all_words)
        new_data.append(bag)
    preds = model(torch.from_numpy(np.array(new_data)).to(device).float())
    preds = torch.round(preds)
    data = pd.DataFrame({'id':ids,'target':preds.view(-1).cpu().detach().numpy()})
    data.to_csv('./submission.csv',index=False)
    model.train()
    return data

In [9]:
data = pd.read_csv('./data/train.csv')

In [10]:
X = data['text']

In [11]:
y = data['target']

In [12]:
all_words = []
all_data = []
tags = []

In [13]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for X_batch_in_X_batch in X_batch:
        new_X.append(stem(X_batch_in_X_batch))
    all_words.extend(new_X)
    all_data.append((new_X,y_batch))
    tags.append(y_batch)

7613it [00:02, 3073.35it/s]


In [14]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)
np.random.shuffle(tags)

In [15]:
all_words = sorted(set(all_words))

In [16]:
tags = sorted(set(tags))

In [17]:
tags

[0, 1]

In [18]:
np.random.shuffle(all_words)
np.random.shuffle(all_data)
np.random.shuffle(tags)

In [19]:
X = []
y = []

In [20]:
for sentence,tag in tqdm(all_data):
    bag = bag_of_words(sentence,all_words)
    X.append(bag)
    y.append(tags.index(tag))

  2%|▉                                      | 190/7613 [00:00<00:27, 272.27it/s]


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import *

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)

In [ ]:
device = 'cuda'

In [ ]:
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [ ]:
class Model(Module):
    def __init__(self,hidden=1024,activation=ReLU,iters=10):
        super().__init__()
        self.activation = activation()
        self.linear1 = Linear(len(all_words),hidden)
        self.linear2 = Linear(hidden,hidden)
        self.linear3 = Linear(hidden,1)
        self.iters = iters
    
    def forward(self,X):
        preds = self.activation(self.linear1(X))
        for _ in range(self.iters):
            preds = self.activation(self.linear2(preds))
        preds = Sigmoid()(self.linear3(preds))
        return preds

In [ ]:
model = Model().to(device)

In [ ]:
criterion = BCELoss()

In [ ]:
optimizer = Adam(model.parameters(),lr=0.001)

In [ ]:
epochs = 100

In [ ]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds.view(-1,1),y.view(-1,1))
    return loss.item()

In [ ]:
batch_size = 32

In [ ]:
def get_accuracy(model,X,y):
    correct = 0
    total = 0
    preds = model(X)
    for pred,y_batch in zip(preds,y):
        pred = int(torch.round(pred))
        y_batch = int(torch.round(y_batch))
        if pred == y_batch:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [ ]:
PROJECT_NAME = 'Natural-Language-Processing-with-Disaster-Tweets-Competition'

In [ ]:
import wandb

In [ ]:
epochs_iter = tqdm(range(epochs))

In [21]:
wandb.init(project=PROJECT_NAME,name='baseline')
for epoch in epochs_iter:
    torch.cuda.empty_cache()
    for idx in range(0,len(X_train),batch_size):
        torch.cuda.empty_cache()
        X_batch = X_train[idx:idx+batch_size].to(device).float()
        y_batch = y_train[idx:idx+batch_size].to(device).float()
        preds = model(X_batch)
        loss = criterion(preds.view(-1,1),y_batch.view(-1,1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epochs_iter.set_description(f'{epoch}/{idx}')
    epochs_iter.set_description(f'{epoch}/Getting/Loss')
    wandb.log({'Loss':get_loss(model,X_train,y_train,criterion)})
    epochs_iter.set_description(f'{epoch}/Getting/Val/Loss')
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    epochs_iter.set_description(f'{epoch}/Getting/Acc')
    wandb.log({'Acc':get_accuracy(model,X_train,y_train)})
    epochs_iter.set_description(f'{epoch}/Getting/Val/Acc')
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    epochs_iter.set_description(f'{epoch}/Making/Submission')
    make_submission(model)
wandb.finish()

IndentationError: expected an indented block (2437309456.py, line 5)